chroma 는 메모리 데이터베이스여서 데이터가 날아감 -> pinecone 사용할 수 있음

langchain을 사용하면 벡터데이터베이스 부분만 변경하면 간단히 변경 가능

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 200,
)

loader = Docx2txtLoader('./tax.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [5]:
len(document_list)

In [4]:
document_list[1]

Document(metadata={'source': './tax.docx'}, page_content='1. 구성원 간 이익의 분배비율이 정하여져 있고 해당 구성원별로 이익의 분배비율이 확인되는 경우\n\n2. 구성원 간 이익의 분배비율이 정하여져 있지 아니하나 사실상 구성원별로 이익이 분배되는 것으로 확인되는 경우\n\n④ 제3항에도 불구하고 해당 단체의 전체 구성원 중 일부 구성원의 분배비율만 확인되거나 일부 구성원에게만 이익이 분배되는 것으로 확인되는 경우에는 다음 각 호의 구분에 따라 소득세 또는 법인세를 납부할 의무를 진다.<신설 2018. 12. 31.>\n\n1. 확인되는 부분: 해당 구성원별로 소득세 또는 법인세에 대한 납세의무 부담\n\n2. 확인되지 아니하는 부분: 해당 단체를 1거주자 또는 1비거주자로 보아 소득세에 대한 납세의무 부담\n\n⑤ 제3항 및 제4항에도 불구하고 법인으로 보는 단체 외의 법인 아닌 단체에 해당하는 국외투자기구(투자권유를 하여 모은 금전 등을 가지고 재산적 가치가 있는 투자대상자산을 취득, 처분하거나 그 밖의 방법으로 운용하고 그 결과를 투자자에게 배분하여 귀속시키는 투자행위를 하는 기구로서 국외에서 설립된 기구를 말한다. 이하 같다)를 제119조의2제1항제2호에 따라 국내원천소득의 실질귀속자로 보는 경우 그 국외투자기구는 1비거주자로서 소득세를 납부할 의무를 진다.<신설 2018. 12. 31.>\n\n[전문개정 2009. 12. 31.]\n\n[제1조에서 이동, 종전 제2조는 제2조의2로 이동 <2009. 12. 31.>]\n\n\n\n제2조의2(납세의무의 범위) ① 제43조에 따라 공동사업에 관한 소득금액을 계산하는 경우에는 해당 공동사업자별로 납세의무를 진다. 다만, 제43조제3항에 따른 주된 공동사업자(이하 이 항에서 “주된 공동사업자”라 한다)에게 합산과세되는 경우 그 합산과세되는 소득금액에 대해서는 주된 공동사업자의 특수관계인은 같은 조 제2항에 따른 손익분배비율에 해당하는 그의 소득금액을 한도로 주된

In [6]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings

load_dotenv()

embedding = UpstageEmbeddings(model='solar-embedding-1-large')

/Users/choyoungrae/Desktop/lecture-study/51/inflearn-llm-application/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/choyoungrae/Desktop/lecture-study/51/inflearn-llm-application/.venv/lib/python3.12/site-packages/langchain_pinecone/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from l

[3.2 코드](3.2%20LangChain과%20Chroma를%20활용한%20RAG%20구성.ipynb)에서 여기 벡터데이터베이스 부분만 변경하면 됨

In [8]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'tax-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)


database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [9]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs = database.similarity_search(query, k=3) # k는 몇 개를 가져올지

In [10]:
retrieved_docs

In [11]:
from langchain_upstage import ChatUpstage

llm = ChatUpstage()

연봉 5천만원인 직장인의 소득세를 계산하기 위해서는 다음과 같은 과정을 거칩니다.

1. **근로소득공제**: 총급여액인 5천만원에서 근로소득공제액을 차감합니다. 근로소득공제액은 총급여액에 따라 달라지며, 5천만원의 경우 120만원입니다. 따라서 과세표준은 4,880만원이 됩니다.

2. **세율 적용**: 과세표준인 4,880만원에 대해 세율을 적용합니다. 2024년 세율에 따르면, 1,400만원 초과 5,000만원 이하의 소득에 대해서는 15%의 세율이 적용됩니다.

3. **산출세액 계산**: 과세표준에 세율을 곱하여 산출세액을 계산합니다. 4,880만원 x 15% = 732만원입니다.

4. **세액공제 적용**: 근로소득세액공제를 적용합니다. 총급여액이 5천만원인 경우, 공제세액은 50만원입니다.

5. **최종 납부할 소득세**: 산출세액에서 세액공제를 차감하여 최종 납부할 소득세를 계산합니다. 732만원 - 50만원 = 682만원입니다.

따라서, 연봉 5천만원인 직장인의 최종 납부할 소득세는 약 682만원입니다. 하지만, 실제 납부할 소득세는 다른 소득, 공제항목 등에 따라 달라질 수 있으므로, 국세청의 소득세 계산기를 이용하거나 세무 전문가와 상담하는 것이 좋습니다.


In [12]:
prompt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrieved_docs}

Question: {query}
"""

In [13]:
ai_message = llm.invoke(prompt)

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [14]:
print(ai_message.content)

In [15]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [16]:
prompt

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 경우, 소득세는 근로소득세를 계산해야 합니다. 근로소득세는 총급여액에서 근로소득공제를 뺀 후, 세율을 적용하여 계산합니다. 연봉 5천만원의 경우, 근로소득공제로 총급여액에서 약 1,200만원을 공제할 수 있습니다. 그 결과 과세표준이 약 3,800만원이 되고, 이 금액에 기본세율을 적용하면 산출세액이 나옵니다. 이후 세액공제와 세액감면을 적용하면 실제 납부해야 할 소득세를 알 수 있습니다. 정확한 금액은 개인의 상황에 따라 다를 수 있으므로 국세청 홈페이지나 세무사를 통해 계산하는 것이 정확합니다.'}

In [36]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [38]:
ai_message = qa_chain.invoke({"query": query})

In [39]:
ai_message

{'query': '연봉 5천만원인 직장인의 소득세는 얼마인가요?',
 'result': '연봉 5천만원인 직장인의 자녀세액공제는 55만원입니다. 이는 공제대상자녀가 2명인 경우에 해당하는 금액입니다.'}